<a href="https://colab.research.google.com/github/Rajib2805/ICVL_logistics/blob/main/ICVL_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PRODUCTION AND SHIPMENT OPTIMIZATION**

**THE OPTIMIZATION MODEL IS FRAMED WITH THE OBJECTIVE OF HAVING A MONTHLY VIEW AND DERIVE ESTIMATES FOR THE MONTH AHEAD**

**Even though optimization problems are all about contraints in the decision making process, it must be remembered that constraints must be strived to be changed for the better with better operation and managerial decisions. It must be assumed that no constrains are God created and everything is changeable.**

In [5]:
!pip install pulp
import pulp as p

In [6]:
#Defining the problem
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  

# Create problem Variables  
CCs_135 = p.LpVariable('CCs_135', lowBound = 0) 
CCs_105 = p.LpVariable('CCs_105', lowBound = 0) 
TCs = p.LpVariable('TCs', lowBound = 0) 
ROM_mined_perday = p.LpVariable('ROM_mined_perday', lowBound = 0) 
ROM_135_feed_perday = p.LpVariable('ROM_135_feed_perday', lowBound = 0) 
ROM_105_feed_perday = p.LpVariable('ROM_105_feed_perday', lowBound = 0) 
CCp_135 = p.LpVariable('CCp_135', lowBound = 0) 
CCp_105 = p.LpVariable('CCp_105', lowBound = 0) 
TCp = p.LpVariable('TCp', lowBound = 0) 
CCro_135 = p.LpVariable('CCro_135', lowBound = 0) 
CCro_105 = p.LpVariable('CCro_105', lowBound = 0) 
TCro = p.LpVariable('TCro', lowBound = 0) 
CCra_135 = p.LpVariable('CCra_135', lowBound = 0) 
CCra_105 = p.LpVariable('CCra_105', lowBound = 0) 
TCra = p.LpVariable('TCra', lowBound = 0) 
CClera_135 = p.LpVariable('CClera_135', lowBound = 0) 
CClera_105 = p.LpVariable('CClera_105', lowBound = 0)
TClera = p.LpVariable('TClera', lowBound = 0) 
Stock_ROM = p.LpVariable('Stock_ROM', lowBound = 0) 
Stock_CHPP_CC_135 = p.LpVariable('Stock_CHPP_CC_135', lowBound = 0) 
Stock_CHPP_CC_105 = p.LpVariable('Stock_CHPP_CC_105', lowBound = 0) 
Stock_CHPP_TC = p.LpVariable('Stock_CHPP_TC', lowBound = 0) 
Stock_RS_CC_135 = p.LpVariable('Stock_RS_CC_135', lowBound = 0) 
Stock_RS_CC_105 = p.LpVariable('Stock_RS_CC_105', lowBound = 0) 
Stock_RS_TC = p.LpVariable('Stock_RS_TC', lowBound = 0) 
Stock_Port_CC_135 = p.LpVariable('Stock_Port_CC_135', lowBound = 0) 
Stock_Port_CC_105 = p.LpVariable('Stock_Port_CC_105', lowBound = 0) 
Stock_Port_TC = p.LpVariable('Stock_Port_TC', lowBound = 0) 
ccvessel_135 = p.LpVariable('ccvessel_135', lowBound = 0, cat=p.LpInteger) 
ccvessel_105 = p.LpVariable('ccvessel_105', lowBound = 0, cat=p.LpInteger) 
tcvessel = p.LpVariable('tcvessel', lowBound = 0, cat=p.LpInteger) 

#Variables to be fed by user
EjcDemand_CC135= 77000
EjcDemand_CC105= 77000
EjcDemand_TC= 38500

CokingCoalPrice_135= 100
CokingCoalPrice_105= 160
ThermalCoalPrice= 68

MineWorkingDays= 28
ChppWorkingDays= 28

RomMinedPerDay= 20000
ROMMiningPerMonth_Max= 999999999
Yield_CC135= 0.35
Yield_TC135= 0.10
Yield_CC105= 0.25
Yield_TC105= 0.20

MinimumCashFlowRequired= 8000000

RoadHauagePerMonth_Max= 500000
RailPerMonth_Max= 200000
RailPerMonthLease_Max= 30000

VesselCapacity=38500

RomStock_Min= 100000

ChppStock_CC135_min= 4000
ChppStock_CC105_min= 4000
ChppStock_TC_min= 4000

RSStock_CC135_min= 4000
RSStock_CC105_min= 5000
RSStock_TC_min= 4500

PortStock_CC135_min= 4000
PortStock_CC105_min= 500
PortStock_TC_min= 3500

StorageCapacityROM_Output= 100000
StorageCapacityCHPP_Output= 100000
StorageCapacityRS= 150000
StorageCapacityPort= 150000

MiningCostPerMT= 20
MiningCost_Fixed= 100000
ChppCostPerMT= 11
ChppCost_Fixed= 100000
HaulageCostPerMT= 3
RailCostPerMt_CC= 10.75
RailCostPerMt_TC= 8.00
RailCost_Fixed= 100000
PortCostPerMT= 8.5
PortCost_Fixed= 100000

OpeningStockROM= 30000

OpeningStock_CHPP_CC135= 8000
OpeningStock_CHPP_CC105= 8000
OpeningStock_CHPP_TC= 8000

OpeningStock_RS_CC135= 8000
OpeningStock_RS_CC105= 8000
OpeningStock_RS_TC= 8000

OpeningStock_Port_CC135= 200
OpeningStock_Port_CC105= 600
OpeningStock_Port_TC= 400



# Objective Function of profit earned per ton / (-) for loss incurred per ton
Lp_prob+= (CokingCoalPrice_135 * CCs_135 + CokingCoalPrice_105 * CCs_105 + ThermalCoalPrice * TCs) - ((MiningCostPerMT * (ROM_mined_perday*MineWorkingDays) + MiningCost_Fixed) + (ChppCostPerMT * (CCp_135 + CCp_105) + ChppCost_Fixed) + HaulageCostPerMT * (CCro_135 + CCro_105 + TCro) + (RailCostPerMt_CC * (CCra_135 + CCra_105 + CClera_135 + CClera_105)) + (RailCostPerMt_TC*(TCra+TClera) + RailCost_Fixed) + (PortCostPerMT * (CCs_135 + CCs_105 + TCs)) + (PortCost_Fixed)) 

# DIFFERENT CONSTRAINTS:
#Sales constraint as per EJC
Lp_prob += CCs_135 <= EjcDemand_CC135 #taking the value >=0 shall give us the best decision without any pressure from EJC
Lp_prob += CCs_105 <= EjcDemand_CC105
Lp_prob += TCs <= EjcDemand_TC #taking the value >=0 shall give us the best decision without any pressure from EJC

#Minimum Cashflow generation requirement (Mining cost + CHPP cost + Haulage cost + Rail Cost + Port cost + Other admistrative fixed cost
Lp_prob += CokingCoalPrice_135 * CCs_135 + CokingCoalPrice_105 * CCs_105 + ThermalCoalPrice * TCs >= MinimumCashFlowRequired

#Mining constraint
Lp_prob += Stock_ROM + ROM_mined_perday*MineWorkingDays >= (ROM_135_feed_perday + ROM_105_feed_perday)*MineWorkingDays
Lp_prob += ROM_mined_perday*MineWorkingDays <= ROMMiningPerMonth_Max

#CHPP constraint of yield and working days
Lp_prob += CCp_135 <= Yield_CC135 * ROM_135_feed_perday * ChppWorkingDays
Lp_prob += CCp_135 >= Yield_CC135 * ROM_135_feed_perday * ChppWorkingDays
Lp_prob += CCp_105 <= Yield_CC105 * ROM_105_feed_perday * ChppWorkingDays
Lp_prob += CCp_105 >= Yield_CC105 * ROM_105_feed_perday * ChppWorkingDays
Lp_prob += TCp <= Yield_TC135/Yield_CC135 * CCp_135 + Yield_TC105/Yield_CC105 * CCp_105
Lp_prob += TCp >= Yield_TC135/Yield_CC135 * CCp_135 + Yield_TC105/Yield_CC105 * CCp_105

#Road haulage constraint
Lp_prob += CCro_135 + CCro_105 + TCro >= 0
Lp_prob += CCro_135 + CCro_105 + TCro <= RoadHauagePerMonth_Max

#Rail transport constraint (Rail transport is sum of own railing + lease railing)
Lp_prob += CCra_135 + CCra_105 + TCra >= 0
Lp_prob += CCra_105 + CCra_105 + TCra <= RailPerMonth_Max
Lp_prob += CClera_135 + CClera_105 + TClera <= RailPerMonthLease_Max #assuming that only one train is taken on lease


#Vessel capacity constraint (Note that <= and >= is done to make the equations EQUAL TO in an indirect way)
Lp_prob += ccvessel_135 <= CCs_135*1/VesselCapacity
Lp_prob += ccvessel_105 <= CCs_105*1/VesselCapacity
Lp_prob += tcvessel <= TCs*1/VesselCapacity
Lp_prob += ccvessel_135 >= CCs_135*1/VesselCapacity
Lp_prob += ccvessel_105 >= CCs_105*1/VesselCapacity
Lp_prob += tcvessel >= TCs*1/VesselCapacity

#Stock flow constraint to avoid stock-out
Lp_prob += Stock_ROM + ROM_mined_perday*MineWorkingDays >= (ROM_135_feed_perday + ROM_105_feed_perday)*ChppWorkingDays

Lp_prob += Stock_CHPP_CC_135 + CCp_135 >= CCro_135
Lp_prob += Stock_CHPP_CC_105 + CCp_105 >= CCro_105
Lp_prob += Stock_CHPP_TC + TCp >= TCro

Lp_prob += Stock_RS_CC_135 + CCro_135 >= CCra_135 + CClera_135
Lp_prob += Stock_RS_CC_105 + CCro_105 >= CCra_105 + CClera_105
Lp_prob += Stock_RS_TC + TCro >= TCra + TClera

Lp_prob += Stock_Port_CC_135 + CCra_135 + CClera_135 >= CCs_135
Lp_prob += Stock_Port_CC_105 + CCra_105 + CClera_105 >= CCs_105
Lp_prob += Stock_Port_TC + TCra + TClera >= TCs

#Minimum stock to be maintained at different stockyards as buffer stock
Lp_prob += Stock_ROM + ROM_mined_perday*MineWorkingDays - (ROM_135_feed_perday + ROM_105_feed_perday)*ChppWorkingDays >= RomStock_Min

Lp_prob += Stock_CHPP_CC_135 + CCp_135 + CCp_105 - CCro_135 >= ChppStock_CC135_min
Lp_prob += Stock_CHPP_CC_105 + CCp_105 - CCro_105 >= ChppStock_CC105_min
Lp_prob += Stock_CHPP_TC + TCp - TCro >= ChppStock_TC_min

Lp_prob += Stock_RS_CC_135 + CCro_135 - CCra_135 - CClera_135 >= RSStock_CC135_min
Lp_prob += Stock_RS_CC_105 + CCro_105 - CCra_105 - CClera_105 >= RSStock_CC105_min
Lp_prob += Stock_RS_TC + TCro - TCra- TClera >= RSStock_TC_min

Lp_prob += Stock_Port_CC_135 + CCra_135 + CClera_135 - CCs_135 >= PortStock_CC135_min
Lp_prob += Stock_Port_CC_105 + CCra_105 + CClera_105 - CCs_105 >= PortStock_CC105_min
Lp_prob += Stock_Port_TC + TCra + TClera - TCs >= PortStock_TC_min

#Storage space constraint
Lp_prob += Stock_ROM + ROM_mined_perday * MineWorkingDays - (ROM_135_feed_perday+ROM_105_feed_perday)* ChppWorkingDays <= StorageCapacityROM_Output # space at Mine
Lp_prob += Stock_CHPP_CC_135 + Stock_CHPP_CC_105 + Stock_CHPP_TC + CCp_135 +CCp_105 + TCp - CCro_135 - CCro_105 - TCro <= StorageCapacityCHPP_Output # space at CHPP  
Lp_prob += Stock_RS_CC_135 + Stock_RS_CC_105 + Stock_RS_TC + CCro_135 + CCro_105 + TCro - CCra_135 - CCra_105 - TCra - CClera_135 - CClera_105 - TClera <= StorageCapacityRS # space at RS
Lp_prob += Stock_Port_CC_135 + Stock_Port_CC_105 + Stock_Port_TC + CCra_135 + CCra_105 + TCra + CClera_135 + CClera_105 + TClera - CCs_135 - CCs_105 - TCs <= StorageCapacityPort # space at Port

# Opening/Present Stock levels
Lp_prob += Stock_ROM <= OpeningStockROM
#Lp_prob += Stock_ROM >= OpeningStockROM
Lp_prob += ROM_mined_perday <= RomMinedPerDay #from regression
Lp_prob += Stock_CHPP_CC_135 <= OpeningStock_CHPP_CC135
#Lp_prob += Stock_CHPP_CC_135 >= OpeningStock_CHPP_CC135
Lp_prob += Stock_CHPP_CC_105 <= OpeningStock_CHPP_CC105
#Lp_prob += Stock_CHPP_CC_105 >= OpeningStock_CHPP_CC105
Lp_prob += Stock_CHPP_TC <= OpeningStock_CHPP_TC
#Lp_prob += Stock_CHPP_TC >= OpeningStock_CHPP_TC
Lp_prob += Stock_RS_CC_135 <= OpeningStock_RS_CC135
Lp_prob += Stock_RS_CC_135 >= OpeningStock_RS_CC135
Lp_prob += Stock_RS_CC_105 <= OpeningStock_RS_CC105
Lp_prob += Stock_RS_CC_105 >= OpeningStock_RS_CC105
Lp_prob += Stock_RS_TC <= OpeningStock_RS_TC
Lp_prob += Stock_RS_TC >= OpeningStock_RS_TC
Lp_prob += Stock_Port_CC_135 <= OpeningStock_Port_CC135
Lp_prob += Stock_Port_CC_135 >= OpeningStock_Port_CC135
Lp_prob += Stock_Port_CC_105 <= OpeningStock_Port_CC105
Lp_prob += Stock_Port_CC_105 >= OpeningStock_Port_CC105
Lp_prob += Stock_Port_TC <= OpeningStock_Port_TC
Lp_prob += Stock_Port_TC >= OpeningStock_Port_TC


# Display the problem 
print(Lp_prob) 

#printing the status of the optimization problem
status = Lp_prob.solve()   # Solver CBC by default
print('The status of the problem is: ', p.LpStatus[status]) 

# Printing the final solution 
print('Profit value is: ', round(p.value(Lp_prob.objective)))
print('ROM mined is: ', round(p.value(ROM_mined_perday*MineWorkingDays)))
print('ROM_135_feed is: ', round(p.value(ROM_135_feed_perday*ChppWorkingDays)))
print('ROM_105_feed is: ', round(p.value(ROM_105_feed_perday*ChppWorkingDays)))
print('Production in CHPP CC_135: ', round(p.value(CCp_135)), 'CC_105: ', round(p.value(CCp_105)),' TC: ', round(p.value(TCp)))
print('Road despatch CC_135: ', round(p.value(CCro_135)), 'CC_105: ', round(p.value(CCro_105)), 'TC: ', round(p.value(TCro)))
print('Rail despatch CC_135: ', round(p.value(CCra_135)), ' CC_105: ', round(p.value(CCra_105)), 'TC: ', round(p.value(TCra)))
print('Lease Rail despatch CC_135: ', round(p.value(CClera_135)), 'CC_105: ', round(p.value(CClera_105)), 'TC: ', round(p.value(TClera)))
print('Number of vessels CC_135: ', p.value(ccvessel_135), 'CC_105: ', p.value(ccvessel_105), 'TC: ', p.value(tcvessel))
print('Shipping/Sale CC_135: ', round(p.value(CCs_135)), 'CC_105: ', round(p.value(CCs_105)), 'TC: ', round(p.value(TCs)))

print('Stock at Mine  ROM: ',round(p.value(Stock_ROM + ROM_mined_perday*MineWorkingDays - (ROM_135_feed_perday + ROM_105_feed_perday)*ChppWorkingDays)))
print('Stock at CHPP  Total_CC:',round(p.value(Stock_CHPP_CC_135 + CCp_135 - CCro_135 + Stock_CHPP_CC_105 + CCp_105 - CCro_105 )), 'CC_135:',round(p.value(Stock_CHPP_CC_135 + CCp_135 - CCro_135)), 'CC_105:',round(p.value(Stock_CHPP_CC_105 + CCp_105 - CCro_105 )), 'Total_TC:', round(p.value(Stock_CHPP_TC + TCp-TCro)))
print('Stock at RS  Total_CC:',round(p.value(Stock_RS_CC_135 + CCro_135 - CCra_135 - CClera_135 + Stock_RS_CC_105 + CCro_105 - CCra_105 - CClera_105)), 'CC_135:',round(p.value(Stock_RS_CC_135 + CCro_135 - CCra_135 - CClera_135)), 'CC_105:',round(p.value(Stock_RS_CC_105 + CCro_105 - CCra_105 - CClera_105)),'Total_TC:', round(p.value(Stock_RS_TC + TCro-TCra-TClera)))
print('Stock at Port  Total_CC:',round(p.value(Stock_Port_CC_135 + CCra_135 + CClera_135 - CCs_135 + Stock_Port_CC_105 + CCra_105 + CClera_105 - CCs_105)), 'CC_135:',round(p.value(Stock_Port_CC_135 + CCra_135 + CClera_135 - CCs_135)), 'CC_105:',round(p.value(Stock_Port_CC_105 + CCra_105 + CClera_105 - CCs_105)), 'Total_TC:', round(p.value(Stock_Port_TC + TCra +TClera - TCs)))

Problem:
MAXIMIZE
-10.75*CClera_105 + -10.75*CClera_135 + -11*CCp_105 + -11*CCp_135 + -10.75*CCra_105 + -10.75*CCra_135 + -3*CCro_105 + -3*CCro_135 + 151.5*CCs_105 + 91.5*CCs_135 + -560*ROM_mined_perday + -8.0*TClera + -8.0*TCra + -3*TCro + 59.5*TCs + -400000.0
SUBJECT TO
_C1: CCs_135 <= 77000

_C2: CCs_105 <= 77000

_C3: TCs <= 38500

_C4: 160 CCs_105 + 100 CCs_135 + 68 TCs >= 8000000

_C5: - 28 ROM_105_feed_perday - 28 ROM_135_feed_perday + 28 ROM_mined_perday
 + Stock_ROM >= 0

_C6: 28 ROM_mined_perday <= 999999999

_C7: CCp_135 - 9.8 ROM_135_feed_perday <= 0

_C8: CCp_135 - 9.8 ROM_135_feed_perday >= 0

_C9: CCp_105 - 7 ROM_105_feed_perday <= 0

_C10: CCp_105 - 7 ROM_105_feed_perday >= 0

_C11: - 0.8 CCp_105 - 0.285714285714 CCp_135 + TCp <= 0

_C12: - 0.8 CCp_105 - 0.285714285714 CCp_135 + TCp >= 0

_C13: CCro_105 + CCro_135 + TCro >= 0

_C14: CCro_105 + CCro_135 + TCro <= 500000

_C15: CCra_105 + CCra_135 + TCra >= 0

_C16: 2 CCra_105 + TCra <= 200000

_C17: CClera_105 + CClera_1